# 04 - Portfolio Construction

This notebook demonstrates state-conditioned portfolio construction.

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.data.synthetic_data import SyntheticDataGenerator
from src.portfolio.baseline import BaselinePortfolio
from src.portfolio.state_conditioned import StateConditionedPortfolio
from src.portfolio.volatility_scaling import VolatilityScaledPortfolio
from src.analysis.performance import PerformanceAnalyzer
from src.visualization.styles import set_publication_style, PlotStyles

set_publication_style()
np.random.seed(42)

In [ ]:
# Load data
generator = SyntheticDataGenerator(seed=42)
data = generator.generate(n_months=732)

factors = data['factors']
regimes = data['regimes']
if isinstance(regimes, pd.DataFrame):
    regimes = regimes['regime']

# Define evaluation period
TRAINING_END = '1999-12-31'
EVAL_START = '2000-01-01'

## 1. Build Portfolios

In [ ]:
# Baseline portfolio
baseline = BaselinePortfolio(
    factors,
    target_volatility=0.10,
    transaction_cost=0.002,
)
baseline_result = baseline.backtest(start=EVAL_START)

print("Baseline portfolio constructed")

In [ ]:
# State-conditioned portfolio
state_cond = StateConditionedPortfolio(
    factors,
    regimes,
    target_volatility=0.10,
    transaction_cost=0.002,
)

# Fit optimal exposures
state_cond.fit(training_end=TRAINING_END, regularization=0.5)
state_cond_result = state_cond.backtest(start=EVAL_START)

print("\nOptimal Exposures by Regime:")
print(state_cond.summary())

In [ ]:
# Volatility-scaled portfolio
vol_scaled = VolatilityScaledPortfolio(
    factors,
    target_volatility=0.10,
    transaction_cost=0.002,
)
vol_scaled_result = vol_scaled.backtest(start=EVAL_START)

print("Volatility-scaled portfolio constructed")

## 2. Performance Comparison

In [ ]:
# Compare performance
analyzer = PerformanceAnalyzer()

comparison = analyzer.compare_strategies({
    'Baseline': baseline_result.returns['net'],
    'State-Conditioned': state_cond_result.returns['net'],
    'Vol-Scaled': vol_scaled_result.returns['net'],
})

print("Performance Comparison (Out-of-Sample):")
comparison[['mean_return', 'volatility', 'sharpe_ratio', 'max_drawdown', 'skewness']].round(2)

In [ ]:
# Cumulative returns comparison
fig, ax = plt.subplots(figsize=(12, 6))

cum_baseline = np.cumsum(baseline_result.returns['net']) * 100
cum_cond = np.cumsum(state_cond_result.returns['net']) * 100
cum_vol = np.cumsum(vol_scaled_result.returns['net']) * 100

ax.plot(cum_baseline.index, cum_baseline.values, 'b--', 
        label='Baseline', linewidth=1.5, alpha=0.8)
ax.plot(cum_cond.index, cum_cond.values, 'b-', 
        label='State-Conditioned', linewidth=2)
ax.plot(cum_vol.index, cum_vol.values, 'g-', 
        label='Vol-Scaled', linewidth=1.5, alpha=0.8)

ax.set_xlabel('Date')
ax.set_ylabel('Cumulative Return (%)')
ax.set_title('Portfolio Performance Comparison (Out-of-Sample)')
ax.legend()
plt.tight_layout()
plt.show()

## 3. Drawdown Analysis

In [ ]:
# Drawdown comparison
fig, axes = plt.subplots(3, 1, figsize=(12, 8), sharex=True)

strategies = [
    ('Baseline', baseline_result.returns['net']),
    ('State-Conditioned', state_cond_result.returns['net']),
    ('Vol-Scaled', vol_scaled_result.returns['net']),
]

for ax, (name, returns) in zip(axes, strategies):
    dd, _ = analyzer.compute_drawdown_series(returns)
    ax.fill_between(dd.index, 0, -dd.values * 100, alpha=0.7)
    ax.set_ylabel('Drawdown (%)')
    ax.set_title(name)
    ax.set_ylim([-50, 5])
    
    max_dd = dd.max() * 100
    ax.annotate(f'Max: {max_dd:.1f}%', xy=(0.02, 0.85), xycoords='axes fraction')

axes[-1].set_xlabel('Date')
plt.tight_layout()
plt.show()

## 4. Regime-Specific Analysis

In [ ]:
# Performance by regime
regime_perf = state_cond.analyze_by_regime()
print("State-Conditioned Performance by Regime:")
regime_perf.round(2)

In [ ]:
# Compare vs baseline in each regime
comparison_result = state_cond.compare_with_baseline(baseline_result)
print("\nImprovement vs Baseline:")
print(f"Sharpe improvement: {comparison_result['sharpe_improvement']:.2f}")
print(f"Drawdown reduction: {comparison_result['drawdown_reduction']:.1f}%")

## 5. Key Takeaways

1. **State conditioning improves risk-adjusted returns** by reducing exposure in high-risk states
2. **Maximum drawdown is significantly reduced** compared to unconditional portfolios
3. **Crash-Spike protection** is the primary driver of improvement
4. **Quality and Low-Risk** factors maintain exposure even in stress periods